In [1]:
import numpy as np
import pandas as pd


def which(self):
    try:
        self = list(iter(self))
    except TypeError as e:
        raise Exception("""'which' method can only be applied to iterables.
        {}""".format(str(e)))
    indices = [i for i, x in enumerate(self) if bool(x) == True]
    return(indices)

cb = pd.read_csv("codebook-reworked.csv")
vartype = cb.vartype
vartype = vartype[which(vartype != 0)]
index = 0
numeric = []
categorical = []
for i in vartype:
    if i in [1,2,4,5]:
        numeric.append(index)
    if i == 3:
        categorical.append(index)
    index = index + 1

In [2]:
df = pd.read_csv("train.csv")
group = df["personid"]
df = df.drop(['uniqueid', 'personid'], axis=1)
df = df.dropna(axis=1, how='all')

X = df.drop(["health"], axis=1)
colnames_train = X.columns

test = pd.read_csv("test.csv")
test = test.dropna(axis=1, how='all')
uniqueid = test['uniqueid']
test = test.drop(['uniqueid', 'personid'], axis=1)
colnames_test = test.columns

rm_train = set(colnames_train) - set(colnames_test)
rm_test = set(colnames_test) - set(colnames_train)

X = X.drop(list(rm_train), axis=1)
test = test.drop(list(rm_test), axis=1)

y = df["health"]

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer

# creating a pipeline to process data.
# numerical data is imputed using knn and scaled to be in [0,1].
# categorical data is imputed by treating NA as a seperate category and one hot encoded.

numeric_pipe = Pipeline([('imputer', KNNImputer(n_neighbors=5)), ('scaler', MinMaxScaler())])
categorical_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value=9999999)), ('ohe', OneHotEncoder())])
t = [('cat', categorical_pipe, categorical), ('num', numeric_pipe, numeric)]
col_transform = ColumnTransformer(transformers=t)
X = col_transform.fit_transform(X)

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

# Testing parameters for neural network.

model = MLPClassifier(random_state=20803652, learning_rate_init=0.01, hidden_layer_sizes=1350, max_iter=1000)
param_grid = dict(alpha=[0.4, 0.45, 0.5])
gkf = GroupKFold(n_splits=5).split(X, y, group)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="neg_log_loss", cv=gkf, n_jobs=-1, verbose=3)
grid_result = grid.fit(X, y)
print(grid_result.best_score_)
print(grid_result.best_params_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed: 14.0min remaining: 90.8min
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed: 15.0min remaining: 13.1min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 18.4min finished


-1.2080578882910626
{'hidden_layer_sizes': 1350}
